In [48]:
### DATA-DRIVEN PROGNOSTICS I ###
### GENERATE DATA FOR DATA-DRIVEN MODEL ###

import random
from prog_models.models import BatteryCircuit
import pandas as pd
import warnings

# Ignore warnings when machine exceeds its end of life
warnings.filterwarnings("ignore")

""" Method that uses a physical machine model from the prog_models package and a current (health) state of the model and
an action (i.e., intensity), which is performed for 100 time steps
    Parameter:
        machine             machine model from the prog_models package
        state               current (health) state of the model
        action              loading of the machine for the next 100 time steps
    Return:
        health                               
    """
def produce_model(machine, states, action):
        
        # Define load of battery
        def future_loading(t, x=None):
            return {'i': action}

        # Set current state of machine
        machine.parameters['x0'] = states
        # Simulate 100 steps
        options = {
            'save_freq': 100,  # Frequency at which results are saved
            'dt': 2  # Timestep
        }
        (_, _, states, outputs, event_states) = machine.simulate_to(100, future_loading, **options)
        health = event_states[-1]['EOD']
        return(round(health, 2), states[-1], outputs[-1]['t'], outputs[-1]['v'])
def reset_states(machine):
    # Returns initial states of machine, e.g., {'tb': 18.95, 'qb': 7856.3254, 'qcp': 0, 'qcs': 0} for Battery
    return(machine.default_parameters['x0'])

battery = BatteryCircuit()
states = reset_states(battery)
reset_counter = 0
dataset = []
for i in range(int(1e4)):
    # If asset failed last period, reset all historical values
    if reset_counter == 0: t = v = t_1 = v_1 = t_2 = v_2 = t_3 = v_3 = 0 
    # Shift history by one time period
    v_3 = v_2
    t_3 = t_2
    v_2 = v_1
    t_2 = t_1
    v_1 = v
    t_1 = t

    # Increment reset_counter
    reset_counter = reset_counter + 1
    # Compute new health, states, t, and v using last battery state and a random new action
    health, states, t, v = produce_model(machine=battery, states=states, action=random.sample((0, 1, 2, 3, 4), 1)[0])
    
    if health <= 0: 
        # Reset battery states to initialize battery for next produce_model call
        states = reset_states(battery)
        # Initialize reset_counter
        reset_counter = 0
        # Sometimes produce_model returns weird or negative values as the end of life is exceeded
        # Here, we just simply set it to zero to not confuse a later learner 
        health = 0

    # append to two-dimensional list
    dataset.append([t, v, t_1, v_1, t_2, v_2, t_3, v_3, health])

    # print progress every 10,000 iterations
    if (i+1) % 10000 == 0: print("Iteration", i+1)
# Transform two-dim list to dataframe
dataset = pd.DataFrame(dataset, columns=['t', 'v', 't_1', 'v_1', 't_2', 'v_2', 't_3', 'v_3', 'health'])
# Save it as pickle
dataset.to_pickle('diagnostics/data')

Iteration 10000


In [1]:
### DATA-DRIVEN PROGNOSTICS II ###
### FIT AND TEST MODEL ###
from sklearn import tree, linear_model, kernel_ridge, svm, neighbors, gaussian_process, ensemble, neural_network
import pandas as pd
from sklearn.model_selection import cross_val_score
import pickle

dataset = pd.read_pickle('diagnostics/data')
X = dataset[['t', 'v', 't_1', 'v_1', 't_2', 'v_2', 't_3', 'v_3']]
y = dataset['health']
#learner = [linear_model.LinearRegression(), linear_model.Ridge(), linear_model.Lasso(), linear_model.BayesianRidge(), tree.DecisionTreeRegressor(), # Fast
#        kernel_ridge.KernelRidge(), svm.SVR(), neighbors.KNeighborsRegressor(), gaussian_process.GaussianProcessRegressor(), # Slow
#        ensemble.RandomForestRegressor(), neural_network.MLPRegressor()] # Slow
learner = [ensemble.RandomForestRegressor()]
for i in learner:
    reg = i
    print(i, ":", cross_val_score(reg, X, y, cv=5)) # default scoring R2
# Fit on all data
model = learner[0].fit(X, y)
pickle.dump(model, open('diagnostics/model', 'wb'))

RandomForestRegressor() : [0.92981457 0.94752602 0.94335423 0.94618864 0.93738079]


In [24]:
### DATA-DRIVEN PROGNOSTICS IIIa ###
### FIT, TEST, VISUALIZE MODEL USING TRAIN AND TEST SETS ###

# Find index of healthy machines
index_df = X.index[(X['t_1'] == 0) & (X['v_1'] == 0) & (X['t_2'] == 0) & (X['v_2'] == 0) & (X['t_3'] == 0) & (X['v_3'] == 0)].tolist()
index_test = round(len(index_df)*0.8)

# Create train and test set without disrupting machine runs to-failure
X_train = X.iloc[0:(index_df[index_test])]
y_train = y.iloc[0:(index_df[index_test])]
X_test = X.iloc[index_df[index_test]:(len(X))]
y_test = y.iloc[index_df[index_test]:(len(y))]

## Train
learner = [ensemble.RandomForestRegressor()]
model = learner[0].fit(X_train, y_train)
## Predict
y_pred = pd.DataFrame(model.predict(X_test), columns=['Pred'])
## Analyze
#reset index of each DataFrame
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
# Concat dataframes
test_df = pd.concat([X_test, y_test, y_pred], axis=1)
# Print for visualization (e.g., in R)
test_df.to_excel("diagnostics/test_results.xlsx") 

In [8]:
### REINFORCEMENT LEARNING I ###
### TRAIN, SAVE, EVALUATE MODEL ###

import gym
import stable_baselines3 as sb
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
import pickle

# Load diagnostics model from disk
diag_model = pickle.load(open('diagnostics/model', 'rb'))
# Initiate environment
env = gym.make('Production-v0')
# Callback for best model
best_callback = EvalCallback(env, best_model_save_path='./callback/',
                             log_path='./callback/', eval_freq=1000,
                             deterministic=True, render=False)

#model = sb.DQN('MlpPolicy', env, tensorboard_log="./tensorboard/", gamma = 0.99, learning_rate=0.01)
#model.learn(total_timesteps=2e6, tb_log_name="DQN_DIAG_model", callback = best_callback)
#model.save("DQN_DIAG_model")

model = sb.PPO('MlpPolicy', env, tensorboard_log="./tensorboard/")
model.learn(total_timesteps=2e6, tb_log_name="PPO_TRUE_model", callback = best_callback)
model.save("PPO_TRUE_model")

#model = sb.A2C('MlpPolicy', env, tensorboard_log="./tensorboard/")
#model.learn(total_timesteps=2e6, tb_log_name="A2C_DIAG_model", callback = best_callback)
#model.save("A2C_DIAG_model")

# Evaluate the agent
evaluate_policy(model, model.get_env(), n_eval_episodes=10)

Eval num_timesteps=1000, episode_reward=-1453.40 +/- 13.43
Episode length: 100.00 +/- 0.00
New best mean reward!
Eval num_timesteps=2000, episode_reward=-1479.80 +/- 22.83
Episode length: 100.00 +/- 0.00
Eval num_timesteps=3000, episode_reward=-2292.40 +/- 97.87
Episode length: 100.00 +/- 0.00
Eval num_timesteps=4000, episode_reward=-2248.60 +/- 107.93
Episode length: 100.00 +/- 0.00
Eval num_timesteps=5000, episode_reward=-2219.60 +/- 67.72
Episode length: 100.00 +/- 0.00
Eval num_timesteps=6000, episode_reward=-2317.80 +/- 72.88
Episode length: 100.00 +/- 0.00
Eval num_timesteps=7000, episode_reward=-2098.80 +/- 79.07
Episode length: 100.00 +/- 0.00
Eval num_timesteps=8000, episode_reward=-2082.40 +/- 68.62
Episode length: 100.00 +/- 0.00
Eval num_timesteps=9000, episode_reward=-3157.60 +/- 53.09
Episode length: 100.00 +/- 0.00
Eval num_timesteps=10000, episode_reward=-3183.60 +/- 51.49
Episode length: 100.00 +/- 0.00
Eval num_timesteps=11000, episode_reward=-3129.60 +/- 75.84
Episod

(471.3, 20.65453945262397)

In [9]:
### REINFORCEMENT LEARNING II ###
### LOAD MODEL ###
import gym
import stable_baselines3 as sb
from stable_baselines3 import DQN
from stable_baselines3 import A2C
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
import pickle

diag_model = pickle.load(open('diagnostics/model', 'rb'))
env = gym.make('Production-v0')
# Best Model
model = PPO.load('./callback/best_PPO_truehealth', env = env)
# Last Model
#model = DQN.load('DQN_1_model', env = env)

# Evaluate the agent
evaluate_policy(model, model.get_env(), n_eval_episodes=10)

(460.1, 23.847222060441336)

In [10]:
### REINFORCEMENT LEARNING III ###
### TRY AND EVALUATE MY MODEL ###
import pandas as pd

# Initilaize Reward
result_df = pd.DataFrame([[0, 0, 0, 0, 0, 0]], columns=['RM', 'PM', 'Inventory', 'Spare Parts Inventory', 'Reward', 'Upper'])
# Set iterations
iterations = 1000
for i in range(iterations):
    # Initialize episode
    store = []
    obs = env.reset()
    done = False
    store.append([0, obs[0], env.breakdown, obs[2], obs[3], 0, done, obs[1]])
    # Compute one episode
    while not done:
        # Get best action for state
        action, _state = model.predict(obs, deterministic=True)
        # Compute next state
        obs, reward, done, info = env.step(action)
        # Store results of this episode
        store.append([action, obs[0], env.breakdown, obs[2], obs[3], reward, done, obs[1]])
    eps_df = pd.DataFrame(store, columns=['action', 'health', 'breakdown', 'inventory', 'sp_inventory', 'reward', 'done', 'next_order'])
    # Calculate nr. of reactive maintenance interventions by counting health 'resets' and substracting PM actions
    result_df.iloc[0]['RM'] = result_df.iloc[0]['RM'] + sum(eps_df['breakdown']==True)
    # Calculate nr. of preventive maintenance interventions
    result_df.iloc[0]['PM'] = result_df.iloc[0]['PM'] + sum(eps_df['action']==10)
    # Calculate inventory
    result_df.iloc[0]['Inventory'] = result_df.iloc[0]['Inventory'] + sum(eps_df['inventory'])
    # Calculate spare parts inventory per period
    result_df.iloc[0]['Spare Parts Inventory'] = result_df.iloc[0]['Spare Parts Inventory'] + sum(eps_df['sp_inventory'])
    # Calculate reward
    result_df.iloc[0]['Reward'] = result_df.iloc[0]['Reward'] + sum(eps_df['reward'])
    # Calculate reward with no costs and fulfillment of all orders
    result_df.iloc[0]['Upper'] = result_df.iloc[0]['Upper'] + sum(eps_df.iloc[:-1]['next_order']) * env.order_r

print("The average number of reactive maintenance interventions per episode is: ", result_df.iloc[0]['RM']/iterations)
print("The average number of preventive maintenance interventions per episode is: ", result_df.iloc[0]['PM']/iterations) 
print("The average sum of inventory per episode is: ", result_df.iloc[0]['Inventory']/iterations)
print("The average sum of spare parts inventory per episode is: ", result_df.iloc[0]['Spare Parts Inventory']/iterations)
print("The average reward per episode is: ", result_df.iloc[0]['Reward']/iterations)
print("The average upper bound per episode is: ", result_df.iloc[0]['Upper']/iterations)


The average number of reactive maintenance interventions per episode is:  0.004
The average number of preventive maintenance interventions per episode is:  3.996
The average sum of inventory per episode is:  36.937
The average sum of spare parts inventory per episode is:  10.485
The average reward per episode is:  461.476
The average upper bound per episode is:  643.408


In [5]:
### REINFORCEMENT LEARNING IIIa1 ###
### TRAIN REACTIVE MODEL ###
import gym
import stable_baselines3 as sb
from stable_baselines3.common.callbacks import EvalCallback
import pickle

# Initiate environment
env = gym.make('Production-v0', reactive_mode = True)
# Callback for best model
best_callback = EvalCallback(env, best_model_save_path='./callback/',
                             log_path='./callback/', eval_freq=1000,
                             deterministic=True, render=False)

model = sb.PPO('MlpPolicy', env, tensorboard_log="./tensorboard/")
model.learn(total_timesteps=2e6, tb_log_name="PPO_REACT_model", callback = best_callback)
model.save("PPO_REACT_model")

Eval num_timesteps=1000, episode_reward=-2693.40 +/- 18.94
Episode length: 100.00 +/- 0.00
New best mean reward!
Eval num_timesteps=2000, episode_reward=-2707.20 +/- 20.17
Episode length: 100.00 +/- 0.00
Eval num_timesteps=3000, episode_reward=-1183.60 +/- 21.67
Episode length: 100.00 +/- 0.00
New best mean reward!
Eval num_timesteps=4000, episode_reward=-1216.00 +/- 30.07
Episode length: 100.00 +/- 0.00
Eval num_timesteps=5000, episode_reward=-1285.00 +/- 4.00
Episode length: 100.00 +/- 0.00
Eval num_timesteps=6000, episode_reward=-1273.00 +/- 24.07
Episode length: 100.00 +/- 0.00
Eval num_timesteps=7000, episode_reward=-1319.20 +/- 37.16
Episode length: 100.00 +/- 0.00
Eval num_timesteps=8000, episode_reward=-1323.60 +/- 15.24
Episode length: 100.00 +/- 0.00
Eval num_timesteps=9000, episode_reward=-1198.00 +/- 27.66
Episode length: 100.00 +/- 0.00
Eval num_timesteps=10000, episode_reward=-1224.20 +/- 61.60
Episode length: 100.00 +/- 0.00
Eval num_timesteps=11000, episode_reward=-1213

In [7]:
### REINFORCEMENT LEARNING IIIa2 ###
### EVALUATE REACTIVE MODEL ###

import pandas as pd
from stable_baselines3 import DQN
from stable_baselines3 import PPO
import numpy as np

env = gym.make('Production-v0', reactive_mode = True)
# Best Model
model = PPO.load('./callback/best_PPO_reactive', env = env)
# Initilaize Reward
result_df = pd.DataFrame([[0, 0, 0, 0, 0, 0, 0]], columns=['RM', 'PM', 'MTBF', 'Inventory', 'Spare Parts Inventory', 'Reward', 'Upper'])
# Set iterations
iterations = 1000
for i in range(iterations):
    # Initialize episode
    store = []
    obs = env.reset()
    done = False
    store.append([0, obs[0], env.breakdown, obs[2], obs[3], 0, done, obs[1]])
    # Compute one episode
    while not done:
        # Get best action for state
        action, _state = model.predict(obs, deterministic=True)
        # Compute next state
        obs, reward, done, info = env.step(action)
        # Store results of this episode
        store.append([action, obs[0], env.breakdown, obs[2], obs[3], reward, done, obs[1]])
    eps_df = pd.DataFrame(store, columns=['action', 'health', 'breakdown', 'inventory', 'sp_inventory', 'reward', 'done', 'next_order'])
    # Calculate nr. of reactive maintenance interventions by counting health 'resets' and substracting PM actions
    result_df.iloc[0]['RM'] = result_df.iloc[0]['RM'] + sum(eps_df['breakdown']==True)
    # Calculate nr. of preventive maintenance interventions
    result_df.iloc[0]['PM'] = result_df.iloc[0]['PM'] + sum(eps_df['action']==10)
    # Calculate mean time between failures
    # Cut df after last breakdown
    eps_df_trim = eps_df.iloc[:(np.where(eps_df['breakdown'].eq(True), eps_df.index, 0).max()+1)]
    # Calculate MTBF by dividing periods where machine is running / breakdowns
    #result_df.iloc[0]['MTBF'] = result_df.iloc[0]['MTBF'] + (len(eps_df_trim) -
    #    sum(eps_df_trim['breakdown'] == True)) / sum(eps_df_trim['breakdown'] == True)
    # Calculate inventory
    result_df.iloc[0]['Inventory'] = result_df.iloc[0]['Inventory'] + sum(eps_df['inventory'])
    # Calculate spare parts inventory per period
    result_df.iloc[0]['Spare Parts Inventory'] = result_df.iloc[0]['Spare Parts Inventory'] + sum(eps_df['sp_inventory'])
    # Calculate reward
    result_df.iloc[0]['Reward'] = result_df.iloc[0]['Reward'] + sum(eps_df['reward'])
    # Calculate reward with no costs and fulfillment of all orders
    result_df.iloc[0]['Upper'] = result_df.iloc[0]['Upper'] + sum(eps_df.iloc[:-1]['next_order']) * env.order_r

print("The average number of reactive maintenance interventions per episode is: ", result_df.iloc[0]['RM']/iterations)
print("The average number of preventive maintenance interventions per episode is: ", result_df.iloc[0]['PM']/iterations)
print("The average mean time between failure per episode is: ", result_df.iloc[0]['MTBF']/iterations)
print("The average sum of inventory per episode is: ", result_df.iloc[0]['Inventory']/iterations)
print("The average sum of spare parts inventory per episode is: ", result_df.iloc[0]['Spare Parts Inventory']/iterations)
print("The average reward per episode is: ", result_df.iloc[0]['Reward']/iterations)
print("The average upper bound per episode is: ", result_df.iloc[0]['Upper']/iterations)

The average number of reactive maintenance interventions per episode is:  0.0
The average number of preventive maintenance interventions per episode is:  0.0
The average mean time between failure per episode is:  0.0
The average sum of inventory per episode is:  27.901
The average sum of spare parts inventory per episode is:  0.0
The average reward per episode is:  -908.089
The average upper bound per episode is:  644.052


21.5

In [57]:
### REINFORCEMENT LEARNING IIIb1 ###
### TRAIN TIME-BASED PREVENTIVE MODEL ###
import gym
import stable_baselines3 as sb
from stable_baselines3.common.callbacks import EvalCallback
import pickle

# Initiate environment
env = gym.make('Production-v0', reactive_mode = True)
# Callback for best model
best_callback = EvalCallback(env, best_model_save_path='./callback/',
                             log_path='./callback/', eval_freq=1000,
                             deterministic=True, render=False)

model = sb.DQN('MlpPolicy', env, tensorboard_log="./tensorboard/")
model.learn(total_timesteps=2e6, tb_log_name="DQN_TIME_model", callback = best_callback)
model.save("DQN_TIME_model")

In [18]:
### REINFORCEMENT LEARNING IIIb2 ###
### EVALUATE TIME-BASED PREVENTIVE MODEL ###
test = 1
test2 = 0
if test or test2: print("true")
else: print("false")

true


In [37]:
### REINFORCEMENT LEARNING IV ###
### VISUALIZE STATE-ACTION ###
import numpy as np
state_action = []

# Define observation grid
grid_health = np.arange(0.0, 1.01, 0.01)
grid_order = range(0, 5)
grid_inventory = range(0, 10)
grid_sp_inventory = [0, 1]

# Loop through grid and store best action for each state
for hlt in grid_health:
    for ord in grid_order:
        for inv in grid_inventory:
            for sin in grid_sp_inventory:
                # Predict
                action, _state = model.predict((hlt, ord, inv, sin), deterministic=True)
                state_action.append([hlt, ord, inv, sin, action])

state_action_df = pd.DataFrame(state_action, columns=['health', 'order', 'inventory', 'sp_inventory', 'action'])
state_action_df.to_excel("visuals/state_action.xlsx") 